In [1]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import time
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from tests import test_prediction, test_generation

In [2]:
# load all that we need

dataset = np.load('../dataset/wiki.train.npy', allow_pickle=True)
devset = np.load('../dataset/wiki.valid.npy', allow_pickle=True)
fixtures_pred = np.load('../fixtures/prediction.npz')  # dev
fixtures_gen = np.load('../fixtures/generation.npy')  # dev
fixtures_pred_test = np.load('../fixtures/prediction_test.npz')  # test
fixtures_gen_test = np.load('../fixtures/generation_test.npy')  # test
vocab = np.load('../dataset/vocab.npy')

In [3]:
# data loader

class LanguageModelDataLoader(DataLoader):
    """
        TODO: Define data loader logic here
    """
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset=dataset
        self.batch_size=batch_size
        self.shuffle=shuffle
        
        #raise NotImplemented


    def __iter__(self):        
        # concatenate your articles and build into batches
        if self.shuffle:
            np.random.shuffle(self.dataset)
        
        #Concatenate all text in one long string
        total_data=np.concatenate(self.dataset)
        num_batches=(len(total_data)-1)//self.batch_size
        
        #input sequences
        inp_seq=total_data[:num_batches*self.batch_size]
        
        #output sequences
        out_seq=total_data[1:(num_batches*self.batch_size)+1]
        
        #grouping into batches
        inp_seq=torch.from_numpy(inp_seq.reshape(self.batch_size, -1)).type(torch.LongTensor)
        out_seq=torch.from_numpy(out_seq.reshape(self.batch_size, -1)).type(torch.LongTensor)
        
        #looping to return tuple
        i=0
        while i<num_batches:
            probability=np.random.uniform()
            if probability<0.95:
                seq_length=int(np.random.normal(70,5))
            else:
                seq_length=int(np.random.normal(35,5))
            x1=inp_seq[:,i:i+seq_length]
            y1=out_seq[:,i:i+seq_length]
            i+=seq_length
            yield x1,y1
        

        
        

In [4]:
# model

class LanguageModel(nn.Module):
    """
        TODO: Define your model here
    """
    def __init__(self, vocab_size):
        super(LanguageModel, self).__init__()
        self.embedding=nn.Embedding(vocab_size, 400)
        self.lstm=nn.LSTM(input_size=400, hidden_size=1150, num_layers=3, batch_first=True)
        self.fc=nn.Linear(1150, vocab_size)


    def forward(self, x):
        # Feel free to add extra arguments to forward (like an argument to pass in the hiddens)
        x=self.embedding(x)
        if h_0:
            x,h_n=self.lstm(x, h_0)
        else:
            x,h_n=self.lstm(x)
        x=self.fc(x)
        
        return x,h_n
        
        

    


In [5]:
# model trainer

class LanguageModelTrainer:
    def __init__(self, model, loader, max_epochs=1, run_id='exp'):
        """
            Use this class to train your model
        """
        # feel free to add any other parameters here
        self.model = model
        self.loader = loader
        self.train_losses = []
        self.val_losses = []
        self.predictions = []
        self.predictions_test = []
        self.generated_logits = []
        self.generated = []
        self.generated_logits_test = []
        self.generated_test = []
        self.epochs = 0
        self.max_epochs = max_epochs
        self.run_id = run_id
        
        # TODO: Define your optimizer and criterion here
        self.optimizer = torch.optim.AdamW(self.model.parameters(),0.001)
        self.criterion = nn.CrossEntropyLoss()

    def train(self):
        self.model.train() # set to training mode
        epoch_loss = 0
        num_batches = 0
        for batch_num, (inputs, targets) in enumerate(self.loader):
            epoch_loss += self.train_batch(inputs, targets)
        epoch_loss = epoch_loss / (batch_num + 1)
        self.epochs += 1
        print('[TRAIN]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, epoch_loss))
        self.train_losses.append(epoch_loss)

    def train_batch(self, inputs, targets):
        """ 
            TODO: Define code for training a single batch of inputs
        
        """
        self.optimizer.zero_grad()
        inputs=inputs.cuda()
        targets=targets.cuda()
        outp,h_n = self.model(inputs)
        outp=outp.reshape((outp.shape[0]*outp.shape[1],outp.shape[2]))
        targets=targets.reshape((targets.shape[0]*targets.shape[1]))
        loss = self.criterion(outp,targets)
        loss.backward()
        self.optimizer.step()
        return loss

    
    def test(self):
        # don't change these
        self.model.eval() # set to eval mode
        predictions = TestLanguageModel.prediction(fixtures_pred['inp'], self.model) # get predictions
        self.predictions.append(predictions)
        generated_logits = TestLanguageModel.generation(fixtures_gen, 10, self.model) # generated predictions for 10 words
        generated_logits_test = TestLanguageModel.generation(fixtures_gen_test, 10, self.model)
        nll = test_prediction(predictions, fixtures_pred['out'])
        generated = test_generation(fixtures_gen, generated_logits, vocab)
        generated_test = test_generation(fixtures_gen_test, generated_logits_test, vocab)
        self.val_losses.append(nll)
        
        self.generated.append(generated)
        self.generated_test.append(generated_test)
        self.generated_logits.append(generated_logits)
        self.generated_logits_test.append(generated_logits_test)
        
        # generate predictions for test data
        predictions_test = TestLanguageModel.prediction(fixtures_pred_test['inp'], self.model) # get predictions
        self.predictions_test.append(predictions_test)
            
        print('[VAL]  Epoch [%d/%d]   Loss: %.4f'
                      % (self.epochs + 1, self.max_epochs, nll))
        return nll

    def save(self):
        # don't change these
        model_path = os.path.join('experiments', self.run_id, 'model-{}.pkl'.format(self.epochs))
        torch.save({'state_dict': self.model.state_dict()},
            model_path)
        np.save(os.path.join('experiments', self.run_id, 'predictions-{}.npy'.format(self.epochs)), self.predictions[-1])
        np.save(os.path.join('experiments', self.run_id, 'predictions-test-{}.npy'.format(self.epochs)), self.predictions_test[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-{}.npy'.format(self.epochs)), self.generated_logits[-1])
        np.save(os.path.join('experiments', self.run_id, 'generated_logits-test-{}.npy'.format(self.epochs)), self.generated_logits_test[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated[-1])
        with open(os.path.join('experiments', self.run_id, 'generated-{}-test.txt'.format(self.epochs)), 'w') as fw:
            fw.write(self.generated_test[-1])


In [6]:
class TestLanguageModel:
    def prediction(inp, model):
        """
            TODO: write prediction code here
            
            :param inp:
            :return: a np.ndarray of logits
        """
        inp=torch.cuda.LongTensor(inp)
        outp,h_n= model(inp)
        pred = outp[:, -1]
        return pred.cpu().detach().numpy()

        
    def generation(inp, forward, model):
        """
            TODO: write generation code here

            Generate a sequence of words given a starting sequence.
            :param inp: Initial sequence of words (batch size, length)
            :param forward: number of additional words to generate
            :return: generated words (batch size, forward)
        """ 
        model.eval()
        with torch.no_grad():
            result=[]
            pred=torch.cuda.LongTensor(inp)
            for i in range(forward):
                    if i==0:
                        h_n=None
                    outp,h_n=model(pred, h_n)
                    pred=torch.argmax(outp, dim=2)
                    result.append(pred)
            return torch.cat(result, dim=1).cpu().detach().numpy()
    
        
        

In [7]:
# TODO: define other hyperparameters here

NUM_EPOCHS = 10
BATCH_SIZE = 100


In [8]:
run_id = str(int(time.time()))
if not os.path.exists('./experiments'):
    os.mkdir('./experiments')
os.mkdir('./experiments/%s' % run_id)
print("Saving models, predictions, and generated words to ./experiments/%s" % run_id)

Saving models, predictions, and generated words to ./experiments/1638242760


In [10]:
model = LanguageModel(len(vocab))
model=model.cuda()
loader = LanguageModelDataLoader(dataset=dataset, batch_size=BATCH_SIZE, shuffle=True)
trainer = LanguageModelTrainer(model=model, loader=loader, max_epochs=NUM_EPOCHS, run_id=run_id)

AssertionError: Torch not compiled with CUDA enabled

In [32]:
best_nll = 1e30 
for epoch in range(NUM_EPOCHS):
    trainer.train()
    nll = trainer.test()
    if nll < best_nll:
        best_nll = nll
        print("Saving model, predictions and generated output for epoch "+str(epoch)+" with NLL: "+ str(best_nll))
        trainer.save()
    

------------------------
torch.Size([20, 64, 33278]) torch.Size([20, 64])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 62, 33278]) torch.Size([20, 62])
------------------------
torch.Size([20, 34, 33278]) torch.Size([20, 34])
------------------------
torch.Size([20, 79, 33278]) torch.Size([20, 79])
------------------------
torch.Size([20, 62, 33278]) torch.Size([20, 62])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([2

------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 64, 33278]) torch.Size([20, 64])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 74, 33278]) torch.Size([20, 74])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 76, 33278]) torch.Size([20, 76])
------------------------
torch.Size([2

------------------------
torch.Size([20, 37, 33278]) torch.Size([20, 37])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 63, 33278]) torch.Size([20, 63])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 64, 33278]) torch.Size([20, 64])
------------------------
torch.Size([20, 80, 33278]) torch.Size([20, 80])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 62, 33278]) torch.Size([20, 62])
------------------------
torch.Size([2

------------------------
torch.Size([20, 74, 33278]) torch.Size([20, 74])
------------------------
torch.Size([20, 68, 33278]) torch.Size([20, 68])
------------------------
torch.Size([20, 68, 33278]) torch.Size([20, 68])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 77, 33278]) torch.Size([20, 77])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 58, 33278]) torch.Size([20, 58])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([2

------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 72, 33278]) torch.Size([20, 72])
------------------------
torch.Size([20, 80, 33278]) torch.Size([20, 80])
------------------------
torch.Size([20, 79, 33278]) torch.Size([20, 79])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 72, 33278]) torch.Size([20, 72])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([2

...

------------------------
torch.Size([20, 40, 33278]) torch.Size([20, 40])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 72, 33278]) torch.Size([20, 72])
------------------------
torch.Size([20, 74, 33278]) torch.Size([20, 74])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 80, 33278]) torch.Size([20, 80])
------------------------
torch.Size([20, 58, 33278]) torch.Size([20, 58])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([2

------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 60, 33278]) torch.Size([20, 60])
------------------------
torch.Size([20, 64, 33278]) torch.Size([20, 64])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 76, 33278]) torch.Size([20, 76])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 68, 33278]) torch.Size([20, 68])
------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([2

------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 74, 33278]) torch.Size([20, 74])
------------------------
torch.Size([20, 71, 33278]) torch.Size([20, 71])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([20, 77, 33278]) torch.Size([20, 77])
------------------------
torch.Size([20, 70, 33278]) torch.Size([20, 70])
------------------------
torch.Size([20, 59, 33278]) torch.Size([20, 59])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 74, 33278]) torch.Size([20, 74])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 65, 33278]) torch.Size([20, 65])
------------------------
torch.Size([2

------------------------
torch.Size([20, 69, 33278]) torch.Size([20, 69])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 73, 33278]) torch.Size([20, 73])
------------------------
torch.Size([20, 72, 33278]) torch.Size([20, 72])
------------------------
torch.Size([20, 61, 33278]) torch.Size([20, 61])
------------------------
torch.Size([20, 75, 33278]) torch.Size([20, 75])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 66, 33278]) torch.Size([20, 66])
------------------------
torch.Size([20, 76, 33278]) torch.Size([20, 76])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([20, 68, 33278]) torch.Size([20, 68])
------------------------
torch.Size([20, 62, 33278]) torch.Size([20, 62])
------------------------
torch.Size([20, 67, 33278]) torch.Size([20, 67])
------------------------
torch.Size([2

KeyboardInterrupt: 

In [ ]:
# Don't change these
# plot training curves
plt.figure()
plt.plot(range(1, trainer.epochs + 1), trainer.train_losses, label='Training losses')
plt.plot(range(1, trainer.epochs + 1), trainer.val_losses, label='Validation losses')
plt.xlabel('Epochs')
plt.ylabel('NLL')
plt.legend()
plt.show()

In [ ]:
# see generated output
print (trainer.generated[-1]) # get last generated output